<a href="https://colab.research.google.com/github/ErikHartman/bopep/blob/main/bopep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# bopep: identifying peptide binders in large scale peptidomic data

Bayesian optimization guided search for binders in large scale peptidomic datasets.

Relies on ESM2 for peptide embeddings, ColabFold utilizing AlphaFold 2 multimer for docking and PyRosetta for interface energy calculations. A deep ensemble is used as a surrogate model utilizing Torch.

In [ ]:
#@title Installation

import os

print("Fetching bopep")
!git clone https://github.com/ErikHartman/bopep /content/bopep/

print("Installing ColabFold")
!pip install --quiet colabfold
!pip install --quiet biopython

print("Installing esm-fair and fetching ESM model")
!pip install --quiet fair-esm

# Code to fetch ESM model if it doesn't already exist
esm_model_path = "/root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt"
if not os.path.exists(esm_model_path):
    print("Downloading ESM model...")
    import esm
    model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
else:
    print("ESM model already exists.")

print("Installing PyRosetta")
!pip install pyrosettacolabsetup
import pyrosettacolabsetup
pyrosettacolabsetup.install_pyrosetta(serialization=True, cache_wheel_on_google_drive=False)

print("Installing other necessary packages")
!pip install -r https://raw.githubusercontent.com/ErikHartman/bopep/main/requirements.txt


Fetching bopep
Cloning into '/content/bopep'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 8), reused 16 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 21.04 KiB | 742.00 KiB/s, done.
Resolving deltas: 100% (8/8), done.
Installing ColabFold
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.0/230.0 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all th

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


Installing PyRosetta

Note that USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE.
See https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.md or email license@uw.edu for details.

Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels.serialization...

Resolving west.rosettacommons.org (west.rosettacommons.org)... 128.95.160.153, 2607:4000:406::160:153

HTTP request sent, awaiting response... 302 Found
Location: https://west.rosettacommons.org/pyrosetta/release/release/PyRosetta4.Release.python310.ubuntu.cxx11thread.serialization.wheel/pyrosetta-2024.42+release.3366cf78a3-cp310-cp310-linux_x86_64.whl [following]
--2024-11-13 04:04:20--  https://west.rosettacommons.org/pyrosetta/release/release/PyRosetta4.Release.python310.ubuntu.cxx11thread.serialization.wheel/pyrosetta-2024.42+release.3366cf78a3-cp310-cp310-linux_x86_64.whl

HTTP request sent, awaiting response... 200 OK
Length: 1576918002 (1.5G)
--2024-11-13 04:04:20--  ht

In [4]:
#@title Embedding settings
import pandas as pd

%cd bopep

# Data input
data_file = "data/test_data.csv" #@param {type:"string"}
data = pd.read_csv(data_file)  # Load the CSV file
peptides = data["peptide"].tolist()

# Set maximum and minimum peptide length
max_length = 30  #@param {type:"slider", min:1, max:100, step:1}
min_length = 5   #@param {type:"slider", min:1, max:100, step:1}

# Set maximum repeat length for amino acids
max_repeat_length = 5  #@param {type:"slider", min:1, max:20, step:1}

# Set maximum allowed fraction of single amino acids
max_single_aa_fraction = 0.75  #@param {type:"number"}

# Display settings
print(f"Data file: {data_file}")
print(f"Max length: {max_length}")
print(f"Min length: {min_length}")
print(f"Max repeat length: {max_repeat_length}")
print(f"Max single amino acid fraction: {max_single_aa_fraction}")

[Errno 2] No such file or directory: 'bopep'
/content/bopep
Data file: data/test_data.csv
Max length: 30
Min length: 5
Max repeat length: 5
Max single amino acid fraction: 0.75


In [2]:
#@title Generate embeddings
from src.embeddings.embed import embed
from src.embeddings.utils import filter_peptides

filtered_peptides = filter_peptides(peptides, max_single_aa_fraction, max_repeat_length, min_length, max_length)
emeddings = embed(filtered_peptides, model_path=esm_model_path)


ModuleNotFoundError: No module named 'tqdm'

In [ ]:
#@title Bayesian optimization settings

In [ ]:
#@title Initialize PyRosetta

In [ ]:
#@title Run bopep!